In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about Pikachu",
        "answer": '''
            Name: Pikachu,
            Type: Electric,
            Abilities: Static, Lightning Rod,
            Description: Pikachu is a species of Pokémon, fictional creatures that appear in an assortment of video games, animated television shows and movies, trading card games, and comic books licensed by The Pokémon Company, a Japanese corporation.,
            Evolution: Pichu -> Pikachu -> Raichu,
            Appearance: Red cheeks that store electricity, long ears with black tips, and a lightning bolt-shaped tail
        '''
    },
    {
        "question": "What do you know about Charizard",
        "answer": '''
            Name: Charizard,
            Type: Fire/Flying,
            Abilities: Blaze, Solar Power,
            Description: Charizard is a species of Pokémon known for its dragon-like appearance and ability to fly. It is one of the final forms of Charmander.,
            Evolution: Charmander -> Charmeleon -> Charizard,
            Appearance: Large wings, fire at the tip of its tail
        '''
    },
    {
        "question": "What do you know about Bulbasaur",        
        "answer": '''
            Name: Bulbasaur,
            Type: Grass/Poison,
            Abilities: Overgrow, Chlorophyll,
            Description: Bulbasaur is a dual-type Grass/Poison Pokémon known for the plant bulb on its back. It evolves into Ivysaur.,
            Evolution: Bulbasaur -> Ivysaur -> Venusaur,
            Appearance: Plant bulb on its back, large eyes
        '''
    }
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

final_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {pokemon}?",
    input_variables=["pokemon"]
)

final_prompt.format(pokemon="Mew")

'Human: What do you know about Charizard\nAI: \n            Name: Charizard,\n            Type: Fire/Flying,\n            Abilities: Blaze, Solar Power,\n            Description: Charizard is a species of Pokémon known for its dragon-like appearance and ability to fly. It is one of the final forms of Charmander.,\n            Evolution: Charmander -> Charmeleon -> Charizard,\n            Appearance: Large wings, fire at the tip of its tail\n        \n\nHuman: What do you know about Mew?'